In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from itertools import combinations
from sklearn.preprocessing import StandardScaler

C:\Users\kykyk\AppData\Local\Temp\ipykernel_12800\512973339.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


#### Data Analysis

In [2]:
from datetime import datetime
health_df = pd.read_csv('healthcare_dataset.csv')
health_df.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import chi2_contingency
from itertools import product
import scipy.stats as ss

# Dropping unnecessary columns for prediction (e.g., Name, Doctor, Date of Admission, etc.)
health_df.drop_duplicates()
health_df['Discharge Date'] = pd.to_datetime(health_df['Discharge Date'])
health_df['Date of Admission'] = pd.to_datetime(health_df['Date of Admission'])
health_df['Inpatient Days']= (health_df['Discharge Date'] - health_df['Date of Admission']).dt.days
health_df["Per Day Amount"] = health_df["Billing Amount"]/health_df["Inpatient Days"]
features = health_df.drop(columns=['Name', 'Doctor', 'Date of Admission', 'Discharge Date'])

label_encoded_columns = ['Hospital', 'Gender', 'Insurance Provider']
label_encoder = LabelEncoder()

for col in label_encoded_columns:
    features[col] = label_encoder.fit_transform(features[col])


ohe_columns = ['Blood Type', 'Medical Condition', 'Admission Type', 'Medication', 'Test Results']
ohe = OneHotEncoder()

array_hot_encoded = ohe.fit_transform(features[ohe_columns])
encoded_df = pd.DataFrame(array_hot_encoded.toarray(), columns=ohe.get_feature_names_out(['Blood Type', 'Medical Condition', 'Admission Type', 'Medication', 'Test Results']))
df = features.join(encoded_df)

# Drop the original columns
df.drop(['Blood Type', 'Medical Condition', 'Admission Type', 'Medication', 'Test Results'], axis=1, inplace=True)
df = df[df['Billing Amount'] > 0]
df = df[df['Inpatient Days'] >= 0]
scaler = StandardScaler()
#df['Age'] = scaler.fit_transform(df[['Age']])
#df['Inpatient Days'] = scaler.fit_transform(df[['Inpatient Days']])

print(df.columns)

Index(['Age', 'Gender', 'Hospital', 'Insurance Provider', 'Billing Amount',
       'Room Number', 'Inpatient Days', 'Per Day Amount', 'Blood Type_A+',
       'Blood Type_A-', 'Blood Type_AB+', 'Blood Type_AB-', 'Blood Type_B+',
       'Blood Type_B-', 'Blood Type_O+', 'Blood Type_O-',
       'Medical Condition_Arthritis', 'Medical Condition_Asthma',
       'Medical Condition_Cancer', 'Medical Condition_Diabetes',
       'Medical Condition_Hypertension', 'Medical Condition_Obesity',
       'Admission Type_Elective', 'Admission Type_Emergency',
       'Admission Type_Urgent', 'Medication_Aspirin', 'Medication_Ibuprofen',
       'Medication_Lipitor', 'Medication_Paracetamol', 'Medication_Penicillin',
       'Test Results_Abnormal', 'Test Results_Inconclusive',
       'Test Results_Normal'],
      dtype='object')


In [8]:
y = df['Billing Amount']
a =df.drop(columns = ['Billing Amount'])
#a = a.drop(columns = ['Per Day Amount'])

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, root_mean_squared_error, mean_squared_error

clf = LinearRegression()

X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2,random_state=2)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
mse = mean_squared_error(y_train, ytrainpred)
target_range = np.max(y_train) - np.min(y_train)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_train, ytrainpred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(rmse_accuracy)
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
mse = mean_squared_error(y_test, ypred)
target_range = np.max(y_test) - np.min(y_test)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_test, ypred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(rmse_accuracy)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))


-328466.4012121726
74.95067518114872
0.1447872526991274
0.4874274246819045
-324089.80033810233
75.20538199483991
0.1471070565600031
0.4890594670284686


In [19]:
from sklearn.inspection import permutation_importance

result = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=2)
importance_scores = result.importances_mean
feature_importances = list(zip(X_test.columns, importance_scores))
feature_importances.sort(key=lambda x: x[1], reverse=True)

for feature, importance in feature_importances:
    print(f"{feature}: {importance:.4f}")

Medication_Ibuprofen: 71623860178393.7812
Medication_Lipitor: 71463196637202.7188
Medication_Penicillin: 71121786192211.4531
Medication_Aspirin: 70663894760206.1406
Medication_Paracetamol: 70603646194989.7188
Medical Condition_Hypertension: 39156653564780.8281
Medical Condition_Asthma: 39106278592143.1094
Medical Condition_Arthritis: 39106278544095.3047
Medical Condition_Cancer: 38433769182269.4531
Medical Condition_Obesity: 38300274987181.8281
Medical Condition_Diabetes: 38272568690268.1719
Test Results_Abnormal: 35196322455916.6406
Test Results_Normal: 35025164434888.8242
Test Results_Inconclusive: 34066113482293.2617
Blood Type_A+: 28463737961860.3008
Blood Type_O+: 27882155467005.1094
Blood Type_AB-: 27625176854202.7695
Blood Type_B+: 27059373724162.6875
Blood Type_B-: 26946663801221.6797
Blood Type_A-: 26867767076749.5117
Blood Type_AB+: 26809157851268.8398
Blood Type_O-: 26770836416240.5234
Admission Type_Urgent: 8662950070132.1348
Admission Type_Emergency: 8637658840428.7598
Adm

In [35]:
b = a.drop(columns = ['Room Number', 'Hospital', 'Insurance Provider', 'Age', 'Gender'])
X_train, X_test, y_train, y_test = train_test_split(b, y, test_size=0.2,random_state=2)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
mse = mean_squared_error(y_train, ytrainpred)
target_range = np.max(y_train) - np.min(y_train)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_train, ytrainpred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_train, ytrainpred))
print(rmse_accuracy)
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
mse = mean_squared_error(y_test, ypred)
target_range = np.max(y_test) - np.min(y_test)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_test, ypred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_test, ypred))
print(rmse_accuracy)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

-328478.12120829895
13117.010679294144
74.95022842918198
0.1447567471755905
0.48741828300611945
-324107.7977578955
13075.369778055921
75.20469376690167
0.14705970814766656
0.48904528477530784


In [36]:
from sklearn.linear_model import Lasso

clf = Lasso(alpha=0.51)
X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2, random_state=42)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
mse = mean_squared_error(y_train, ytrainpred)
target_range = np.max(y_train) - np.min(y_train)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_train, ytrainpred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_train, ytrainpred))
print(rmse_accuracy)
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
mse = mean_squared_error(y_test, ypred)
target_range = np.max(y_test) - np.min(y_test)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_test, ypred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_test, ypred))
print(rmse_accuracy)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

-324817.19848056976
13092.371494926369
75.18270860199135
0.1450330947812426
0.48841671985279345
-331455.0884362018
13170.758428604657
74.82642398814936
0.14632811981842708
0.4851744443234207


In [37]:
from sklearn.inspection import permutation_importance

result = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=2)
importance_scores = result.importances_mean
feature_importances = list(zip(X_test.columns, importance_scores))
feature_importances.sort(key=lambda x: x[1], reverse=True)

for feature, importance in feature_importances:
    print(f"{feature}: {importance:.4f}")

Per Day Amount: 0.4092
Inpatient Days: 0.1112
Blood Type_O+: 0.0001
Medication_Penicillin: 0.0001
Room Number: 0.0000
Blood Type_AB+: 0.0000
Gender: 0.0000
Insurance Provider: 0.0000
Blood Type_A+: 0.0000
Medical Condition_Arthritis: 0.0000
Medication_Ibuprofen: 0.0000
Blood Type_A-: 0.0000
Medication_Aspirin: 0.0000
Medical Condition_Asthma: 0.0000
Admission Type_Emergency: 0.0000
Medication_Paracetamol: 0.0000
Test Results_Abnormal: 0.0000
Medical Condition_Cancer: -0.0000
Medical Condition_Diabetes: -0.0000
Age: -0.0000
Test Results_Normal: -0.0000
Admission Type_Elective: -0.0000
Medication_Lipitor: -0.0000
Medical Condition_Hypertension: -0.0000
Blood Type_B+: -0.0000
Admission Type_Urgent: -0.0000
Blood Type_O-: -0.0000
Hospital: -0.0000
Blood Type_AB-: -0.0001
Medical Condition_Obesity: -0.0001
Blood Type_B-: -0.0001
Test Results_Inconclusive: -0.0002


In [38]:
b = a[['Per Day Amount','Inpatient Days']]
X_train, X_test, y_train, y_test = train_test_split(b, y, test_size=0.2,random_state=2)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
mse = mean_squared_error(y_train, ytrainpred)
target_range = np.max(y_train) - np.min(y_train)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_train, ytrainpred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_train, ytrainpred))
print(rmse_accuracy)
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
mse = mean_squared_error(y_test, ypred)
target_range = np.max(y_test) - np.min(y_test)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_test, ypred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_test, ypred))
print(rmse_accuracy)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

-328650.5414561626
13120.451781223454
74.94365690008982
0.1443079611971041
0.4872838128910604
-323816.4605399726
13069.493611526013
75.21583695834167
0.1478261710563351
0.4892749115504481


In [45]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2,random_state=42)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
mse = mean_squared_error(y_train, ytrainpred)
target_range = np.max(y_train) - np.min(y_train)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_train, ytrainpred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_train, ytrainpred))
print(rmse_accuracy)
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
mse = mean_squared_error(y_test, ypred)
target_range = np.max(y_test) - np.min(y_test)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_test, ypred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_test, ypred))
print(rmse_accuracy)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

98.64736549300697
26.712971520874632
99.94936413173143
0.9999964407616964
0.9989561929556899
94.77419968002093
52.28888017609086
99.90005905075074
0.9999865448640657
0.9979561046587949


In [46]:
from sklearn.inspection import permutation_importance

result = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=2)
importance_scores = result.importances_mean
feature_importances = list(zip(X_test.columns, importance_scores))
feature_importances.sort(key=lambda x: x[1], reverse=True)

for feature, importance in feature_importances:
    print(f"{feature}: {importance:.4f}")

Per Day Amount: 2.7321
Inpatient Days: 1.3562
Hospital: 0.0000
Room Number: 0.0000
Medical Condition_Cancer: 0.0000
Insurance Provider: 0.0000
Medical Condition_Asthma: 0.0000
Medication_Penicillin: 0.0000
Test Results_Abnormal: 0.0000
Admission Type_Urgent: 0.0000
Blood Type_A-: 0.0000
Medical Condition_Obesity: 0.0000
Test Results_Inconclusive: 0.0000
Admission Type_Elective: 0.0000
Test Results_Normal: 0.0000
Blood Type_B-: 0.0000
Blood Type_O-: 0.0000
Blood Type_AB+: 0.0000
Medical Condition_Arthritis: 0.0000
Blood Type_A+: 0.0000
Medication_Aspirin: 0.0000
Blood Type_AB-: 0.0000
Blood Type_O+: 0.0000
Medication_Paracetamol: 0.0000
Blood Type_B+: -0.0000
Medication_Ibuprofen: -0.0000
Age: -0.0000
Gender: -0.0000
Medical Condition_Diabetes: -0.0000
Medication_Lipitor: -0.0000
Medical Condition_Hypertension: -0.0000
Admission Type_Emergency: -0.0000


In [47]:
b = a[['Per Day Amount','Inpatient Days']]
X_train, X_test, y_train, y_test = train_test_split(b, y, test_size=0.2,random_state=2)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
mse = mean_squared_error(y_train, ytrainpred)
target_range = np.max(y_train) - np.min(y_train)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_train, ytrainpred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_train, ytrainpred))
print(rmse_accuracy)
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
mse = mean_squared_error(y_test, ypred)
target_range = np.max(y_test) - np.min(y_test)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_test, ypred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_test, ypred))
print(rmse_accuracy)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

99.6523238190963
13.492829122118417
99.97423252175233
0.999999095047148
0.9994727321881778
97.40483323329805
36.993454414426466
99.92984794721713
0.9999931725198632
0.9985543827603848


In [48]:
from sklearn.neighbors import KNeighborsRegressor
clf = KNeighborsRegressor(n_neighbors=49)
X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2,random_state=42)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
mse = mean_squared_error(y_train, ytrainpred)
target_range = np.max(y_train) - np.min(y_train)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_train, ytrainpred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_train, ytrainpred))
print(rmse_accuracy)
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
mse = mean_squared_error(y_test, ypred)
target_range = np.max(y_test) - np.min(y_test)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_test, ypred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_test, ypred))
print(rmse_accuracy)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

-167121.7829427756
9392.439249226396
82.1961283426402
0.5599830020807676
0.6329912513126691
-177383.65161782084
9636.339963466815
81.58183996302591
0.5430237451860715
0.6233296584040517


In [49]:
from sklearn.inspection import permutation_importance

result = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=2)
importance_scores = result.importances_mean
feature_importances = list(zip(X_test.columns, importance_scores))
feature_importances.sort(key=lambda x: x[1], reverse=True)

for feature, importance in feature_importances:
    print(f"{feature}: {importance:.4f}")

Per Day Amount: 1.0831
Hospital: 0.0026
Inpatient Days: 0.0008
Insurance Provider: 0.0000
Gender: 0.0000
Admission Type_Urgent: 0.0000
Test Results_Inconclusive: 0.0000
Medication_Ibuprofen: 0.0000
Blood Type_AB+: 0.0000
Medication_Aspirin: 0.0000
Admission Type_Elective: 0.0000
Admission Type_Emergency: 0.0000
Blood Type_AB-: 0.0000
Blood Type_A+: 0.0000
Blood Type_B-: 0.0000
Medication_Penicillin: 0.0000
Medical Condition_Asthma: 0.0000
Medication_Lipitor: 0.0000
Blood Type_B+: 0.0000
Blood Type_O+: 0.0000
Medical Condition_Obesity: -0.0000
Medical Condition_Hypertension: -0.0000
Blood Type_O-: -0.0000
Medical Condition_Arthritis: -0.0000
Blood Type_A-: -0.0000
Medical Condition_Cancer: -0.0000
Test Results_Normal: -0.0000
Test Results_Abnormal: -0.0000
Medical Condition_Diabetes: -0.0000
Age: -0.0000
Medication_Paracetamol: -0.0000
Room Number: -0.0006


In [51]:
b = a[['Per Day Amount','Hospital','Inpatient Days']]
X_train, X_test, y_train, y_test = train_test_split(b, y, test_size=0.2,random_state=2)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
mse = mean_squared_error(y_train, ytrainpred)
target_range = np.max(y_train) - np.min(y_train)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_train, ytrainpred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_train, ytrainpred))
print(rmse_accuracy)
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
mse = mean_squared_error(y_test, ypred)
target_range = np.max(y_test) - np.min(y_test)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_test, ypred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_test, ypred))
print(rmse_accuracy)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

-168521.96580383158
9396.640788889887
82.05508015128112
0.5611004226288916
0.6328015286938
-174140.97597213413
9585.557996832044
81.82255263285461
0.5415990919216309
0.6254189258371168


In [52]:
from sklearn import ensemble
params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}

clf = ensemble.GradientBoostingRegressor(**params)
X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2,random_state=42)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
mse = mean_squared_error(y_train, ytrainpred)
target_range = np.max(y_train) - np.min(y_train)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_train, ytrainpred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_train, ytrainpred))
print(rmse_accuracy)
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
mse = mean_squared_error(y_test, ypred)
target_range = np.max(y_test) - np.min(y_test)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_test, ypred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_test, ypred))
print(rmse_accuracy)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

-5577.731997058113
1730.6905181715538
96.71938342676279
0.9850599691955777
0.9323734181734009
-5920.110259711091
1774.7417182473275
96.60789499800775
0.9844997135512313
0.9306279591845885


In [53]:
from sklearn.inspection import permutation_importance

result = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=2)
importance_scores = result.importances_mean
feature_importances = list(zip(X_test.columns, importance_scores))
feature_importances.sort(key=lambda x: x[1], reverse=True)

for feature, importance in feature_importances:
    print(f"{feature}: {importance:.4f}")

Per Day Amount: 2.8166
Inpatient Days: 1.4111
Age: 0.0000
Gender: 0.0000
Hospital: 0.0000
Insurance Provider: 0.0000
Room Number: 0.0000
Blood Type_A+: 0.0000
Blood Type_A-: 0.0000
Blood Type_AB+: 0.0000
Blood Type_AB-: 0.0000
Blood Type_B+: 0.0000
Blood Type_B-: 0.0000
Blood Type_O+: 0.0000
Blood Type_O-: 0.0000
Medical Condition_Arthritis: 0.0000
Medical Condition_Asthma: 0.0000
Medical Condition_Cancer: 0.0000
Medical Condition_Diabetes: 0.0000
Medical Condition_Hypertension: 0.0000
Medical Condition_Obesity: 0.0000
Admission Type_Elective: 0.0000
Admission Type_Emergency: 0.0000
Admission Type_Urgent: 0.0000
Medication_Aspirin: 0.0000
Medication_Ibuprofen: 0.0000
Medication_Lipitor: 0.0000
Medication_Paracetamol: 0.0000
Medication_Penicillin: 0.0000
Test Results_Abnormal: 0.0000
Test Results_Inconclusive: 0.0000
Test Results_Normal: 0.0000


In [54]:
params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}

b = a[['Per Day Amount', 'Inpatient Days']]

clf = ensemble.GradientBoostingRegressor(**params)
X_train, X_test, y_train, y_test = train_test_split(b, y, test_size=0.2,random_state=42)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
mse = mean_squared_error(y_train, ytrainpred)
target_range = np.max(y_train) - np.min(y_train)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_train, ytrainpred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_train, ytrainpred))
print(rmse_accuracy)
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
mse = mean_squared_error(y_test, ypred)
target_range = np.max(y_test) - np.min(y_test)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_test, ypred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_test, ypred))
print(rmse_accuracy)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

-5577.731997058119
1730.6905181715547
96.71938342676278
0.9850599691955777
0.9323734181734008
-5920.110259711096
1774.7417182473284
96.60789499800775
0.9844997135512313
0.9306279591845884


In [55]:
from sklearn.neural_network import MLPRegressor

X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2, random_state=42)
clf = MLPRegressor(hidden_layer_sizes=(40,), max_iter=1000, alpha = 0.005, learning_rate_init= 0.002, activation='relu', solver='adam', random_state=2)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
mse = mean_squared_error(y_train, ytrainpred)
target_range = np.max(y_train) - np.min(y_train)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_train, ytrainpred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_train, ytrainpred))
print(rmse_accuracy)
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
mse = mean_squared_error(y_test, ypred)
target_range = np.max(y_test) - np.min(y_test)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_test, ypred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_test, ypred))
print(rmse_accuracy)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))


-6446.495341297735
1858.3880384971515
96.47732593748731
0.9827739593713349
0.9273836486469136
-6722.984176083227
1889.3834963417191
96.388777509016
0.9824325129271335
0.9261467808658811


In [56]:
from sklearn.inspection import permutation_importance

result = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=2)
importance_scores = result.importances_mean
feature_importances = list(zip(X_test.columns, importance_scores))
feature_importances.sort(key=lambda x: x[1], reverse=True)

for feature, importance in feature_importances:
    print(f"{feature}: {importance:.4f}")

Per Day Amount: 9.7891
Inpatient Days: 7.4228
Admission Type_Urgent: 0.0252
Test Results_Inconclusive: 0.0250
Test Results_Abnormal: 0.0248
Admission Type_Elective: 0.0247
Test Results_Normal: 0.0244
Admission Type_Emergency: 0.0244
Medication_Aspirin: 0.0142
Medication_Paracetamol: 0.0142
Medication_Penicillin: 0.0141
Medication_Lipitor: 0.0141
Medication_Ibuprofen: 0.0135
Medical Condition_Asthma: 0.0113
Medical Condition_Obesity: 0.0112
Medical Condition_Cancer: 0.0111
Medical Condition_Diabetes: 0.0110
Medical Condition_Arthritis: 0.0108
Medical Condition_Hypertension: 0.0106
Blood Type_A-: 0.0077
Blood Type_AB+: 0.0076
Blood Type_AB-: 0.0075
Blood Type_B+: 0.0074
Blood Type_A+: 0.0072
Blood Type_B-: 0.0071
Blood Type_O+: 0.0070
Blood Type_O-: 0.0070
Age: 0.0002
Room Number: 0.0001
Hospital: 0.0000
Gender: 0.0000
Insurance Provider: -0.0000


In [67]:
from sklearn.neural_network import MLPRegressor
b=a[['Per Day Amount','Inpatient Days','Admission Type_Urgent']]

X_train, X_test, y_train, y_test = train_test_split(b, y, test_size=0.2, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
clf = MLPRegressor(hidden_layer_sizes=(400,), max_iter=1000, alpha = 0.005, learning_rate_init= 0.002, activation='relu', solver='adam', random_state=2)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
mse = mean_squared_error(y_train, ytrainpred)
target_range = np.max(y_train) - np.min(y_train)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_train, ytrainpred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_train, ytrainpred))
print(rmse_accuracy)
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
mse = mean_squared_error(y_test, ypred)
target_range = np.max(y_test) - np.min(y_test)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_test, ypred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_test, ypred))
print(rmse_accuracy)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

C:\Users\kykyk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


-3986.792723130616
1468.328659389094
97.21670438222534
0.9892462678396063
0.942625185040348
-4574.884476351588
1563.933840691905
97.01081698297202
0.9879633353257173
0.9388681287459528


In [69]:
df['Billing Amount']

bins = [0, 10000, 20000, 30000, 40000, float('inf')]
labels = [10, 20, 30, 40, 50]

df['cat bill'] = pd.cut(df['Billing Amount'], bins=bins, labels=labels, right=True)

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=42)
df['l'] = kmeans.fit_predict(df[['Billing Amount']])

centroids = kmeans.cluster_centers_

# Find the min and max values within each cluster
cluster_edges = {}
for cluster_label in df['l'].unique():
    # Filter the data for the current cluster
    cluster_data = df[df['l'] == cluster_label]['Billing Amount']
    
    # Find the min and max values for each cluster (edges)
    min_val = cluster_data.min()
    max_val = cluster_data.max()
    
    cluster_edges[cluster_label] = (min_val, max_val)

# Print the cluster edges
print("Cluster edges (min, max) for each cluster:")
for cluster_label, (min_val, max_val) in cluster_edges.items():
    print(f"Cluster {cluster_label}: Min = {min_val}, Max = {max_val}")
print(df['cat bill'])

Cluster edges (min, max) for each cluster:
Cluster 0: Min = 10363.66893067308, Max = 19878.29987681973
Cluster 4: Min = 29703.870474486597, Max = 39836.3848779898
Cluster 2: Min = 19879.862382196294, Max = 29699.31359663662
Cluster 1: Min = 39837.74085278687, Max = 52764.276736469175
Cluster 3: Min = 9.238787497393332, Max = 10362.621045769216
0        20
1        40
2        30
3        40
4        20
         ..
55495    10
55496    40
55497    30
55498    40
55499    10
Name: cat bill, Length: 55392, dtype: category
Categories (5, int64): [10 < 20 < 30 < 40 < 50]


In [73]:
from sklearn.neural_network import MLPClassifier
X_train, X_test, y_train, y_test = train_test_split(a, df['cat bill'], test_size=0.2, random_state=42)
clf = MLPClassifier(hidden_layer_sizes=(400,), max_iter=1000, alpha = 0.1, learning_rate_init= 0.005, activation='relu', solver='adam', random_state=2)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
mse = mean_squared_error(y_train, ytrainpred)
target_range = np.max(y_train) - np.min(y_train)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_train, ytrainpred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_train, ytrainpred))
print(rmse_accuracy)
print(r2_score(y_train, ytrainpred))
print(accuracy_score(y_train, ytrainpred))

ypred = clf.predict(X_test)
mse = mean_squared_error(y_test, ypred)
target_range = np.max(y_test) - np.min(y_test)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_test, ypred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_test, ypred))
print(rmse_accuracy)
print(r2_score(y_test, ypred))
print(accuracy_score(y_test, ypred))

-74.51425089702795
8.354980572018775
79.11254856995306
0.6428871249076868
0.5469275381942094
-72.15001353912807
8.298192900605
79.2545177484875
0.6528942085013247
0.5546529470168787


In [74]:
from sklearn.inspection import permutation_importance

result = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=2)
importance_scores = result.importances_mean
feature_importances = list(zip(X_test.columns, importance_scores))
feature_importances.sort(key=lambda x: x[1], reverse=True)

for feature, importance in feature_importances:
    print(f"{feature}: {importance:.4f}")

Per Day Amount: 0.3545
Inpatient Days: 0.1897
Hospital: 0.0022
Age: 0.0014
Admission Type_Elective: 0.0014
Gender: 0.0012
Test Results_Abnormal: 0.0012
Insurance Provider: 0.0011
Medication_Ibuprofen: 0.0008
Admission Type_Urgent: 0.0007
Test Results_Normal: 0.0005
Medical Condition_Hypertension: 0.0005
Medication_Lipitor: 0.0005
Blood Type_A-: 0.0004
Medication_Paracetamol: 0.0003
Medical Condition_Cancer: 0.0003
Blood Type_B-: 0.0002
Blood Type_O+: 0.0002
Medical Condition_Obesity: 0.0002
Test Results_Inconclusive: 0.0002
Blood Type_B+: 0.0002
Blood Type_A+: 0.0001
Blood Type_AB-: 0.0001
Blood Type_O-: 0.0001
Medical Condition_Asthma: 0.0000
Medication_Aspirin: -0.0001
Room Number: -0.0002
Medical Condition_Arthritis: -0.0002
Medical Condition_Diabetes: -0.0002
Admission Type_Emergency: -0.0002
Blood Type_AB+: -0.0004
Medication_Penicillin: -0.0005


In [79]:
from sklearn.neural_network import MLPClassifier
b= a[['Per Day Amount','Inpatient Days','Hospital','Age','Admission Type_Elective','Gender']]
X_train, X_test, y_train, y_test = train_test_split(b, df['cat bill'], test_size=0.2, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
clf = MLPClassifier(hidden_layer_sizes=(400,20), max_iter=1000, alpha = 0.002, learning_rate_init= 0.00225, activation='relu', solver='adam', random_state=2)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
mse = mean_squared_error(y_train, ytrainpred)
target_range = np.max(y_train) - np.min(y_train)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_train, ytrainpred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_train, ytrainpred))
print(rmse_accuracy)
print(r2_score(y_train, ytrainpred))
print(accuracy_score(y_train, ytrainpred))

ypred = clf.predict(X_test)
mse = mean_squared_error(y_test, ypred)
target_range = np.max(y_test) - np.min(y_test)
mse_accuracy = (1.0 - (mse / target_range)) * 100
rmse = root_mean_squared_error(y_test, ypred)
rmse_accuracy = (1.0 - (rmse / target_range)) * 100
print(mse_accuracy)
print(root_mean_squared_error(y_test, ypred))
print(rmse_accuracy)
print(r2_score(y_test, ypred))
print(accuracy_score(y_test, ypred))

95.67282738699704
1.3156249637344157
96.71093759066396
0.9911451984871883
0.9826913095479882
94.33613141980322
1.5051735554675107
96.23706611133123
0.9885799510202953
0.9773445256792129
